In [1]:
import os 
import csv
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
random.seed(42)
rnd = [random.randrange(189), random.randrange(189), random.randrange(189)]
print(rnd)

[163, 28, 6]


In [3]:
reuse_distances = pd.DataFrame()
test_distances = pd.DataFrame()
tmp_ctr = 0
for file in os.listdir("../ce_data/reuse_distances"):
    x = pd.read_csv("../ce_data/reuse_distances/"+file)
    x["file"] = tmp_ctr
    reuse_distances = pd.concat([reuse_distances, x], ignore_index=True)
    tmp_ctr += 1

In [4]:
cache_misses = pd.read_csv("../ce_data/caches/llc_miss_ratio.csv")

In [5]:
accesses_l1d = pd.read_csv("../ce_data/caches/l1d_accesses.csv")
accesses_l1d = accesses_l1d.sort_values("id", ignore_index=True)
accesses_llc = pd.read_csv("../ce_data/caches/llc_accesses.csv")
accesses_llc = accesses_llc.sort_values("id", ignore_index=True)
# Drop all rows with l1d accesses < llc accesses
for i in ['1024','2048','4096','8192']:
    cache_misses = cache_misses[accesses_l1d[i] >= accesses_llc[i]]
    reuse_distances = reuse_distances[accesses_l1d[i] >= accesses_llc[i]]
print(cache_misses)

<ipython-input-5-c68c1a061e1d>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cache_misses = cache_misses[accesses_l1d[i] >= accesses_llc[i]]
<ipython-input-5-c68c1a061e1d>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  reuse_distances = reuse_distances[accesses_l1d[i] >= accesses_llc[i]]


                         id   32   64  128       256       512      1024  \
0       400.perlbench-41B_0  1.0  1.0  1.0  0.998652  0.889488  0.828841   
1       400.perlbench-41B_1  1.0  1.0  1.0  1.000000  0.928650  0.875960   
2       400.perlbench-41B_2  1.0  1.0  1.0  1.000000  0.950963  0.872154   
3       400.perlbench-41B_3  1.0  1.0  1.0  1.000000  0.946746  0.866864   
4       400.perlbench-41B_4  1.0  1.0  1.0  1.000000  0.956240  0.878444   
...                     ...  ...  ...  ...       ...       ...       ...   
141556     657.xz_s-56B_744  1.0  1.0  1.0  1.000000  1.000000  1.000000   
141557     657.xz_s-56B_745  1.0  1.0  1.0  1.000000  1.000000  1.000000   
141558     657.xz_s-56B_746  1.0  1.0  1.0  1.000000  1.000000  1.000000   
141559     657.xz_s-56B_747  1.0  1.0  1.0  1.000000  1.000000  1.000000   
141560     657.xz_s-56B_748  1.0  1.0  1.0  1.000000  1.000000  1.000000   

            2048      4096      8192  
0       0.803235  0.773585  0.768194  
1       0

In [11]:
test_misses = cache_misses[reuse_distances["file"].isin(rnd)]
test_distances = reuse_distances[reuse_distances["file"].isin(rnd)]
cache_misses = cache_misses[~reuse_distances["file"].isin(rnd)]
reuse_distances = reuse_distances[~reuse_distances["file"].isin(rnd)]

In [14]:
test_distances = test_distances.drop(columns=["file"])
reuse_distances = reuse_distances.drop(columns=["file"])

In [15]:
cache_misses = cache_misses.sort_values("id", ascending=True, ignore_index=True)
reuse_distances = reuse_distances.sort_values("id", ascending=True, ignore_index=True)
test_misses = test_misses.sort_values("id", ascending=True, ignore_index=True)
test_distances = test_distances.sort_values("id", ascending=True, ignore_index=True)

In [16]:
td_ids = test_distances["id"]
test_distances = test_distances.drop(columns=["id"])
tm_ids = test_misses["id"]
test_misses = test_misses.drop(columns=["id",'32','64','128','256','512'])

In [17]:
print(cache_misses)
print(reuse_distances)
print(test_distances)
print(test_misses)

                           id        32        64       128       256  \
0         400.perlbench-41B_0  1.000000  1.000000  1.000000  0.998652   
1         400.perlbench-41B_1  1.000000  1.000000  1.000000  1.000000   
2        400.perlbench-41B_10  1.000000  0.997375  0.997375  0.997375   
3       400.perlbench-41B_100  0.996656  0.996656  0.996656  0.993311   
4       400.perlbench-41B_101  1.000000  1.000000  1.000000  1.000000   
...                       ...       ...       ...       ...       ...   
139163        657.xz_s-56B_95  1.000000  1.000000  1.000000  1.000000   
139164        657.xz_s-56B_96  1.000000  1.000000  1.000000  1.000000   
139165        657.xz_s-56B_97  1.000000  1.000000  1.000000  1.000000   
139166        657.xz_s-56B_98  1.000000  1.000000  1.000000  1.000000   
139167        657.xz_s-56B_99  1.000000  1.000000  1.000000  1.000000   

             512      1024      2048      4096      8192  
0       0.889488  0.828841  0.803235  0.773585  0.768194  
1    

In [18]:
# Quick check if all columns are the same
for i,j in zip(reuse_distances['id'], cache_misses['id']):
    if i!=j:
        print(i, j)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(reuse_distances,cache_misses, test_size=0.1)

In [20]:
X_train_ids = X_train["id"]
X_test_ids = X_test["id"]
y_train_ids = y_train["id"]
y_test_ids = y_test["id"]
print(X_train_ids)

54810          471.omnetpp-188B_23
123245    648.exchange2_s-353B_588
115339        644.nab_s-12459B_212
118151         644.nab_s-7928B_721
32911          450.soplex-247B_744
                    ...           
63211        482.sphinx3-1297B_408
64658        482.sphinx3-1522B_362
133858         654.roms_s-293B_711
41481           458.sjeng-283B_368
60765             481.wrf-455B_219
Name: id, Length: 125251, dtype: object


In [21]:
X_train = X_train.drop(columns=["id"])
X_test = X_test.drop(columns=["id"])
y_train = y_train.drop(columns=["id",'32','64','128','256','512'])
y_test = y_test.drop(columns=["id",'32','64','128','256','512'])

In [22]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(125251, 896) (13917, 896)
(125251, 4) (13917, 4)


In [23]:
print(X_train.columns)

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '3623878656', '3690987520', '3758096384', '3825205248', '3892314112',
       '3959422976', '4026531840', '4093640704', '4160749568', '4227858432'],
      dtype='object', length=896)


In [24]:
from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import MeanSquaredError
from keras.losses import MeanSquaredError
# Create the model
model = Sequential()

# Add the input layer and the first hidden layer
model.add(Dense(1024, input_dim=896, activation='sigmoid'))

# Add the second hidden layer
model.add(Dense(1024, activation='sigmoid'))

# Add the output layer
model.add(Dense(4, activation='sigmoid'))

# Compile the model
model.compile(loss=MeanSquaredError() , optimizer='adam')

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              918528    
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_2 (Dense)             (None, 4)                 4100      
                                                                 
Total params: 1,972,228
Trainable params: 1,972,228
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(X_train.to_numpy(), y_train.to_numpy(), epochs=10)

Epoch 1/10
3915/3915 [==============================] - 46s 12ms/step - loss: 0.1059
Epoch 2/10
3915/3915 [==============================] - 46s 12ms/step - loss: 0.0681
Epoch 3/10
3915/3915 [==============================] - 43s 11ms/step - loss: 0.0659
Epoch 4/10
3915/3915 [==============================] - 42s 11ms/step - loss: 0.0652
Epoch 5/10
3915/3915 [==============================] - 42s 11ms/step - loss: 0.0598
Epoch 6/10
3915/3915 [==============================] - 48s 12ms/step - loss: 0.0603
Epoch 7/10
3915/3915 [==============================] - 46s 12ms/step - loss: 0.0602
Epoch 8/10
3915/3915 [==============================] - 46s 12ms/step - loss: 0.0578
Epoch 9/10
3915/3915 [==============================] - 44s 11ms/step - loss: 0.0581
Epoch 10/10
3915/3915 [==============================] - 48s 12ms/step - loss: 0.0569


In [27]:
f = model(X_test.to_numpy())

In [29]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Compare columns
for i,j in enumerate(['1024','2048','4096','8192']):
    model_out = f.numpy()[:,i]
    y_out = y_test.to_numpy()[:,i]
    print(f'Mean squared error for cache size {j}\t: {mean_squared_error(model_out, y_out)}')

Mean squared error for cache size 1024	: 0.055656720646459136
Mean squared error for cache size 2048	: 0.0611111492296626
Mean squared error for cache size 4096	: 0.06080751771724772
Mean squared error for cache size 8192	: 0.058186930715442654


In [18]:
print(test_distances)

           0      1      2      3      4      5      6      7     8     9  \
0       5648   1600    752    656    608    336    320    208   320   288   
1       5120   1472    976    560    528    336    272    272   240   432   
2       9184   2048   1104   1040    800    496    400    544   384   416   
3      15024   3520   1584   1744   1200    672    752    784   704   608   
4      12976   3072   1440   1440    912    464    560    624   448   544   
...      ...    ...    ...    ...    ...    ...    ...    ...   ...   ...   
2242  184576  66640  35056  28272  20944  23552  16976  13216  8832  6448   
2243  186304  65232  35600  28096  19408  21680  17824  14496  8416  6368   
2244  184192  66160  33264  27136  20144  22800  16416  13152  7712  6256   
2245  186352  66448  35088  28672  20768  22592  17120  14608  8720  6448   
2246  188240  63200  32016  27152  18912  20800  15872  13216  7088  5520   

      ...  3623878656  3690987520  3758096384  3825205248  3892314112  \
0 

In [19]:
f = model(test_distances.to_numpy())

In [20]:
print(f)

tf.Tensor(
[[0.20996214 0.05470732 0.02328947 0.01612861]
 [0.40929827 0.25896218 0.22301589 0.2405568 ]
 [0.18005052 0.07281118 0.03855316 0.02697584]
 ...
 [0.3907446  0.30435568 0.2435273  0.18891862]
 [0.3907446  0.30435568 0.2435273  0.18891862]
 [0.3899918  0.2876871  0.20303065 0.15766022]], shape=(2247, 4), dtype=float32)


In [21]:
print(test_misses)

          1024      2048      4096      8192
0     0.046590  0.000969  0.000962  0.000962
1     0.047124  0.001620  0.001620  0.001620
2     0.049473  0.001464  0.001464  0.001464
3     0.049063  0.003882  0.003766  0.003766
4     0.051722  0.002382  0.002318  0.002318
...        ...       ...       ...       ...
2242  0.000000  0.000000  0.000000  0.000000
2243  0.000000  0.000000  0.000000  0.000000
2244  0.000000  0.000000  0.000000  0.000000
2245  0.000000  0.000000  0.000000  0.000000
2246  0.000000  0.000000  0.000000  0.000000

[2247 rows x 4 columns]


In [22]:
from sklearn.metrics import mean_squared_error
# Compare columns
for i,j in enumerate(['1024','2048','4096','8192']):
    model_out = f.numpy()[:,i]
    y_out = test_misses.to_numpy()[:,i]
    print(f'Mean squared error for cache size {j}\t: {mean_squared_error(model_out, y_out)}')

Mean squared error for cache size 1024	: 0.0728684008729644
Mean squared error for cache size 2048	: 0.06678401400913811
Mean squared error for cache size 4096	: 0.05643301108894797
Mean squared error for cache size 8192	: 0.04919953181115533
